In [28]:
from sklearn.model_selection import train_test_split
from  torch.utils.data import DataLoader 
import numpy as np
import torch 
from torch.utils.data import TensorDataset

In [39]:
train_data_set = np.genfromtxt('train.csv',delimiter=',', dtype=None, encoding='utf-8')
test_data_set=np.genfromtxt('test.csv',delimiter=',', dtype=None, encoding='utf-8')

In [88]:
y=np.delete(train_data_set[:,0],(0),axis=0)
y=y.astype('float') 
x=np.delete(train_data_set[:,1:],(0),axis=0)
x=x.astype('float') 
x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.2,train_size=0.8,shuffle=True) 
inputs = torch.tensor(x_train, dtype=torch.float)
targets = torch.tensor(y_train, dtype=torch.float)
ds = TensorDataset(inputs, targets)
train_loader = DataLoader(dataset=ds, batch_size=64, shuffle=False)

In [89]:
class image_class_NN(torch.nn.Module):
    def __init__(self):
        super(image_class_NN, self).__init__()
        self.flatten = torch.nn.Flatten()
        self.neural_network_graph = torch.nn.Sequential(
        torch.nn.Linear(28*28,300),
        torch.nn.ReLU(),
        torch.nn.Linear(300,200),
        torch.nn.ReLU(),
        torch.nn.Linear(200,10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.neural_network_graph(x)
        return logits
        

In [107]:
model = image_class_NN()
learning_rate = 0.01
epochs = 10
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.set_device(0)


cuda


In [122]:
from torch.optim.lr_scheduler import ExponentialLR
scheduler1 = ExponentialLR(optimizer, gamma=0.9)
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (x, y) in enumerate(dataloader):
        y=y.type(torch.LongTensor)
        x=x.to(device)
        y=y.to(device)
        # Compute prediction and loss
        pred = model(x)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            y=y.type(torch.LongTensor)
            X=X.to(device)
            y=y.to(device)            
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float32).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"validation Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    


In [123]:
epochs = 10
inputs_test = torch.tensor(x_val, dtype=torch.float)
targets_test = torch.tensor(y_val, dtype=torch.float)
ds = TensorDataset(inputs_test, targets_test)
test_dataloader= DataLoader(dataset=ds, batch_size=64, shuffle=False)
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model.to(device), loss_fn, optimizer)
    test_loop(test_dataloader, model.to(device), loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.291318  [   64/33600]
loss: 2.307866  [ 6464/33600]
loss: 2.298510  [12864/33600]
loss: 2.304720  [19264/33600]
loss: 2.295195  [25664/33600]
loss: 2.301915  [32064/33600]
validation Error: 
 Accuracy: 11.0%, Avg loss: 2.301597 

Epoch 2
-------------------------------
loss: 2.291318  [   64/33600]
loss: 2.307866  [ 6464/33600]
loss: 2.298510  [12864/33600]
loss: 2.304720  [19264/33600]
loss: 2.295195  [25664/33600]
loss: 2.301915  [32064/33600]
validation Error: 
 Accuracy: 11.0%, Avg loss: 2.301597 

Epoch 3
-------------------------------
loss: 2.291318  [   64/33600]
loss: 2.307866  [ 6464/33600]
loss: 2.298510  [12864/33600]
loss: 2.304720  [19264/33600]
loss: 2.295195  [25664/33600]
loss: 2.301915  [32064/33600]
validation Error: 
 Accuracy: 11.0%, Avg loss: 2.301597 

Epoch 4
-------------------------------
loss: 2.291318  [   64/33600]
loss: 2.307866  [ 6464/33600]
loss: 2.298510  [12864/33600]
loss: 2.304720  [19264/33600]
loss: 